# EchoCare: Cry Detection Model Training
## Stage 1 of Two-Stage Pipeline

This notebook trains a binary classification model to detect infant cries vs non-cry sounds.

**Dataset:**
- Cry sounds: Baby Chillanto Database (normal, hungry, pain)
- Non-cry sounds: ESC-50 Dataset

**Target Performance:** >85% accuracy for cry detection

**Architecture:** MobileNetV2 (lightweight for Raspberry Pi deployment) with custom classification head

## 1. Import Libraries

In [19]:
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import seaborn as sns
from sklearn.metrics import confusion_matrix
from sklearn.utils.class_weight import compute_class_weight
import cv2
import json
from datetime import datetime

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

print(f"TensorFlow version: {tf.__version__}")
print(f"Keras version: {keras.__version__}")

TensorFlow version: 2.20.0
Keras version: 3.12.0


## 2. Configuration

In [20]:
# Paths
train_dir = Path("C:/Users/danel/FYP/echocare-infant-cry-classification/dataset/processed/mel-spectrograms/cry_detection/train")
val_dir = Path("C:/Users/danel/FYP/echocare-infant-cry-classification/dataset/processed/mel-spectrograms/cry_detection/validate")
test_dir = Path("C:/Users/danel/FYP/echocare-infant-cry-classification/dataset/processed/mel-spectrograms/cry_detection/test")
save_dir = Path("C:/Users/danel/FYP/echocare-infant-cry-classification/model/cry_detection")

# Model hyperparameters
img_size = (224, 224)  # MobileNetV2 input size
batch_size = 32
epochs = 30
learning_rate = 0.0001
dropout_rate = 0.4 

# Class information
class_names = ['non-cry', 'cry']
num_classes = 2